Still To Do 

Add number of times they have been KO'd TKO'd or submitted maybe time since last knock out.

Power Rating (Knockdowns + Knockouts/Technical Knockouts)/(Total Strikes
Landed) 

Total Takedown
Percentage
(Takedowns Landed + (Opponent’s Takedowns Attempted –
Opponent’s Takedowns Landed))/(Takedowns Attempted
+Opponent’s Takedowns Attempted)


Idiot doesnt have win by tko etc correct

In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import sys

from ufc_elo import *
from odds_utils import *
from time_utils import *
from WhoWonAtGraplingStriking import *
from ESPNfeatures import *
from skill import calculate_skill


fight = pd.read_csv('C:/Users/egnke/PythonCode/UFC/Cost-Sensitive-Modeling-UFC/data/data.csv')
fight.date = pd.to_datetime(fight.date)
fight.head()

ModuleNotFoundError: No module named 'trueskill'

In [0]:
! pip install trueskill
from trueskill import Rating, rate_1vs1

def initalise_ratings(df):

  Red_fighters  = list(df['R_fighter'].values)
  Blue_fighters = list(df['B_fighter'].values)
  All_fighters  = Red_fighters + Blue_fighters

  unique_fighters = list(set(All_fighters))
  inital_rating   = [Rating()]*len(unique_fighters)

  return(dict(zip(unique_fighters, inital_rating)))


def compute_update(Red_fighter, Blue_fighter, winner, ratings):

  if winner == 'Red':
    ratings[Red_fighter], ratings[Blue_fighter] = rate_1vs1(ratings[Red_fighter], 
                                                            ratings[Blue_fighter])
  elif winner == 'Blue':
    ratings[Blue_fighter], ratings[Red_fighter] = rate_1vs1(ratings[Blue_fighter], 
                                                            ratings[Red_fighter])
  elif winner == 'Draw':
    ratings[Blue_fighter], ratings[Red_fighter] = rate_1vs1(ratings[Blue_fighter], 
                                                            ratings[Red_fighter],
                                                            drawn=True)
  else:
    pass

  return ratings


def calculate_skill(df, winner_column, red_skill, blue_skill):

  df = df.sort_values(by='date')
  ratings = initalise_ratings(df)
  red_fighters_ratings = []; blue_fighters_ratings =[]

  for index, row in df.iterrows():

    red_fighter = row['R_fighter']; blue_fighter = row['B_fighter']; winner = row[winner_column]
    ratings = compute_update(red_fighter, blue_fighter, winner, ratings)

    red_fighters_ratings.append(ratings[red_fighter].mu)
    blue_fighters_ratings.append(ratings[blue_fighter].mu)

  df[red_skill]  = red_fighters_ratings
  df[blue_skill] = blue_fighters_ratings

  return(df)


In [0]:
def who_won(df, fighter):
  
  if df.R_fighter == fighter and df.Winner == 'Red':
    won = 1
  elif df.B_fighter == fighter and df.Winner == 'Blue':
    won = 1
  else:
    won = 0.5
  return(won)

def calculate_time(last_round_time, last_round):

  if last_round_time >= 5:
    return(last_round*5)
  else:
    return((last_round*5)+last_round_time)

def create_a_merge_column(df, fighter_one, fighter_two, date):

  df['merge'] = df[fighter_one] + df[fighter_two] 
  df['merge'] = df['merge'].apply(lambda x: x.replace(" ", "").replace(".",""))
  df['merge'] = df['merge'].apply(lambda x: ''.join(sorted(x)))
  df['merge'] = df['merge'] + df[date].astype(str)

  return(df)


raw_fight = pd.read_csv('drive/My Drive/UFC/new_total_fight_data.csv', sep=';')
raw_fight.last_round_time = raw_fight.last_round_time.apply(lambda x: float(x.replace(':','.')))
raw_fight['total_fight_time'] = (np.where(raw_fight.last_round_time >= 5, 
                                 raw_fight.last_round*5,
                                 ((raw_fight.last_round-1)*5)+raw_fight.last_round_time))

raw_fight['date'] = pd.to_datetime(raw_fight['date'])
raw_fight = create_a_merge_column(raw_fight, 'R_fighter', 'B_fighter', 'date')

raw_fight_selected = raw_fight[['win_by','total_fight_time','B_GROUND','R_GROUND','B_CLINCH','R_CLINCH','B_DISTANCE',
           'R_DISTANCE','B_LEG','R_LEG','B_BODY','R_BODY','B_HEAD','R_HEAD','B_REV','R_REV',
           'B_PASS','R_PASS','B_SUB_ATT','R_SUB_ATT','B_TD_pct','R_TD_pct','B_TD','R_TD',
           'B_TOTAL_STR.','R_TOTAL_STR.','B_SIG_STR_pct','R_SIG_STR_pct','B_SIG_STR.','R_SIG_STR.',
           'B_KD','R_KD','merge']]

odds = pd.read_csv('drive/My Drive/UFC/raw_fighter_odds_recent.csv')[['Fighter_one','Fighter_two','Average_Odds_f1','Average_Odds_f2','Date']]

odds.Average_Odds_f1 = pd.to_numeric(odds.Average_Odds_f1)
odds.Average_Odds_f2 = pd.to_numeric(odds.Average_Odds_f2)
odds.Date = pd.to_datetime(odds.Date)

# merge the two dataframes
odds = create_a_merge_column(odds, 'Fighter_one', 'Fighter_two', 'Date')
fights = create_a_merge_column(fight, 'R_fighter', 'B_fighter', 'date')
fights_and_odds = fights.merge(odds,on='merge')

# swap columns when needed
fights_and_odds.Average_Odds_f1, fights_and_odds.Average_Odds_f2 = (np.where(fights_and_odds.R_fighter == fights_and_odds.Fighter_one, 
                                                                             [fights_and_odds.Average_Odds_f1, fights_and_odds.Average_Odds_f2], 
                                                                             [fights_and_odds.Average_Odds_f2, fights_and_odds.Average_Odds_f1]))

# convert american odds to probbabilities
fights_and_odds['red_Fighter_Odds'] = fights_and_odds.Average_Odds_f1.apply(lambda x: convert_american_odds_to_perecentage(x))
fights_and_odds['blue_Fighter_Odds']  = fights_and_odds.Average_Odds_f2.apply(lambda x: convert_american_odds_to_perecentage(x))

fights_and_odds = fights_and_odds.merge(raw_fight_selected, on = 'merge')
print(fights_and_odds.shape)

# drop duplicates two odds for same fight
fights_and_odds.drop_duplicates(subset=['merge'], keep='first',inplace=True)
print(fights_and_odds.shape)
fights_and_odds.drop(['merge','Fighter_one','Fighter_two','Date',
                      'Referee','location','no_of_rounds'], inplace = True, axis = 1)

(4462, 185)
(4271, 185)


In [0]:
(fights_and_odds['red_strikes_per_minute'],
fights_and_odds['blue_strikes_per_minute'],
fights_and_odds['red_striking_accuracy'],
fights_and_odds['blue_striking_accuracy'],
fights_and_odds['red_avg_takedowns'],
fights_and_odds['blue_avg_takedowns'],
fights_and_odds['red_td_accuracy'],
fights_and_odds['blue_td_accuracy'],
fights_and_odds['red_td_defense'],
fights_and_odds['blue_td_defense'],
fights_and_odds['red_strikes_absorbed_per_minute'],
fights_and_odds['blue_strikes_absorbed_per_minute'],
fights_and_odds['red_striking_defense'],
fights_and_odds['blue_striking_defense'],
fights_and_odds['red_avg_submissions'],
fights_and_odds['blue_avg_submissions'],
fights_and_odds['red_knockdowns'],
fights_and_odds['blue_knockdowns'],
fights_and_odds['red_power'],
fights_and_odds['blue_power']) = zip(*fights_and_odds.apply(lambda x: ESPN_features(x), axis=1))

FightsOddsElos = calculate_elos(fights_and_odds, k = 25)

FightsOddsElos['Who_Won_at_Wrestling'] = fights_and_odds.apply(lambda x: wrestling(x), axis=1)
FightsOddsElos['Who_Won_at_Striking'] = fights_and_odds.apply(lambda x: striking(x), axis=1)
FightsOddsElos['Who_Won_at_Ground&Pound'] = fights_and_odds.apply(lambda x: ground_and_pound(x), axis=1)
FightsOddsElos['Who_Won_at_JiuJitsu'] = fights_and_odds.apply(lambda x: JiuJitsu(x), axis=1)
FightsOddsElos['Who_Won_at_Grappling'] = fights_and_odds.apply(lambda x: grappling(x), axis=1)

FightsOddsElos = calculate_skill(FightsOddsElos, 'Winner',
                                 'red_skill','blue_skill')
FightsOddsElos = calculate_skill(FightsOddsElos, 'Who_Won_at_Wrestling',
                                 'wrestling_red_skill','wrestling_blue_skill')
FightsOddsElos = calculate_skill(FightsOddsElos, 'Who_Won_at_Striking',
                                 'striking_red_skill','striking_blue_skill')
FightsOddsElos = calculate_skill(FightsOddsElos, 'Who_Won_at_Ground&Pound',
                                 'g_and_p_red_skill','g_and_p_blue_skill')
FightsOddsElos = calculate_skill(FightsOddsElos, 'Who_Won_at_JiuJitsu',
                                 'jiujitsu_red_skill', 'jiujitsu_blue_skill')
FightsOddsElos = calculate_skill(FightsOddsElos, 'Who_Won_at_Grappling',
                                 'grappling_red_skill','grappling_blue_skill')


In [0]:
'''
check output especially takedowns and submissions maybe 
also add how fighter normally wins but would add 16 coumns to dataframe
'''


def extract_stats(row, fighter, red_column, blue_column):
  if row.R_fighter == fighter:
    return(row[red_column])
  elif row.B_fighter == fighter:
    return(row[blue_column])

def list_fighters_attribute(df, fighter, red_column, blue_column):
  values = df.apply(lambda x : extract_stats(x, fighter, red_column, blue_column), axis = 1)
  return(values)

def check_if_red_or_blue(df2, fighter, index):
  if df2.loc[index,'R_fighter'] == fighter:
    return('Red')
  else:
    return('Blue')

def check_if_each_row_is_either_red_or_blue(df,df2):
  blue_or_red = []
  for index, row in df.iterrows():
     blue_or_red.append(check_if_red_or_blue(df2, row['Fighters'], row['Index']))

  df['Blue_or_Red'] = blue_or_red
  return(df)

def who_won(df, fighter):
  
  if df.R_fighter == fighter and df.Winner == 'Red':
    won = 1
  elif df.B_fighter == fighter and df.Winner == 'Blue':
    won = 1
  elif df.Winner == 'Draw' or df.win_by == 'DQ':
    won = 0.5
  else:
    won = 0
  return(won)

def streaks_who_won(df, fighter):
  
  if df.R_fighter == fighter and df.Winner == 'Red':
    won = 1
  elif df.B_fighter == fighter and df.Winner == 'Blue':
    won = 1
  elif df.Winner == 'Draw' or df.win_by == 'DQ':
    won = 0.5
  else:
    won = -1
  return(won)

def calculate_winLoss(df, fighter):

  result  = []
  winLoss = []
  indices   = []
  
  for index, row in df.iterrows():
    # store results
    result.append(who_won(row, fighter))
    # calculate the winLoss ratio and store value for
    # each time iteration
    winLoss.append(sum(result)/len(result))
    indices.append(index)
  return(indices, winLoss)


def calculate_streaks(series):
    df = pd.DataFrame({'index':series.index, 'Column': series.values})
    
    sign = np.sign(df['Column'])
    s = sign.groupby((sign!=sign.shift()).cumsum()).cumsum()
    newdf = df.assign(u_streak=s.where(s>0, 0.0), d_streak=s.where(s<=0, 0.0).abs())
    
    winning_streak = newdf.u_streak.to_list()
    losing_streak  = newdf.d_streak.to_list()
    return(winning_streak,losing_streak)


def remove_nans_at_start_of_carrer(x):

  for index, element in x.items():
    if(np.isnan(element) == True):
      x.loc[index] = 1
    else:
      break
  return(x)


def feature_engineering_fighter_level_loop(df):

  Red_fighters  = list(df['R_fighter'].values)
  Blue_fighters = list(df['B_fighter'].values)
  All_fighters  = Red_fighters + Blue_fighters

  unique_fighters = list(set(All_fighters))

  rank_indexs  = []; fight_number = []
  winLossindex = []; winLossValues = []
  ringRust     = []; fighters = []
  average_fight_time = []; winning_streak = []
  losing_streak = []; takedown_defense = []
  takedown_accuracy = []; strikes_per_minute = []
  striking_accuarcy = []; strikes_absorbed_per_min = []
  striking_defense = []; submission_attempts = []
  average_num_takedowns = []; knockdowns_per_minute = []
  power = []

  df = df.sort_values(by='date')
  for fighter in tqdm(unique_fighters):
    fights = df[(df.R_fighter == fighter) | (df.B_fighter == fighter)]

    fighters = fighters + ([fighter] * len(fights))
    # calculates the number of fights a fighter has had at particular point in time
    rank = fights['date'].rank(ascending=True)
    rank_indexs = rank_indexs + list(rank.index)
    fight_number = fight_number + list(rank.values)

    # calculates the win loss ratio at particular point in time
    indices, winLossratio = calculate_winLoss(fights, fighter)
    winLossindex  = winLossindex  + indices
    winLossValues = winLossValues + winLossratio

    # ring rust feature exapnding standard deviation over time
    ringRust_ = Time_difference_between_consectuive_dates_in_column(fights, 'date').expanding(2).std()
    ringRust  = ringRust + list(ringRust_.values)

    # average fight time
    avg_fight_time = fights['total_fight_time'].expanding(2).mean()
    average_fight_time = average_fight_time + list(avg_fight_time.values)

    # wining streak and losing streak
    wins = fights.apply(lambda x: streaks_who_won(x,fighter),axis=1)
    win_streak, lose_streak = calculate_streaks(wins)

    winning_streak = winning_streak + win_streak
    losing_streak  = losing_streak  + lose_streak

    # Takedown Defense
    td_defense = list_fighters_attribute(fights, fighter, 'red_td_defense', 'blue_td_defense')
    td_defense = remove_nans_at_start_of_carrer(td_defense).expanding(2).mean()
    td_defense = td_defense.to_list()
    
    takedown_defense = takedown_defense + td_defense

    # Takedown Accuarcy
    td_accuarcy = list_fighters_attribute(fights, fighter, 'red_td_accuracy', 'blue_td_accuracy')
    td_accuarcy = remove_nans_at_start_of_carrer(td_accuarcy).expanding(2).mean()
    td_accuarcy = td_accuarcy.to_list()
    
    takedown_accuracy = takedown_accuracy + td_accuarcy

    # strikes per minute
    strike_per_min = list_fighters_attribute(fights, fighter, 'red_strikes_per_minute', 'blue_strikes_per_minute')
    strike_per_min = remove_nans_at_start_of_carrer(strike_per_min).expanding(2).mean()
    strike_per_min = strike_per_min.to_list()
    
    strikes_per_minute = strikes_per_minute + strike_per_min

    # knockdowns per minute
    knockdowns_per_min = list_fighters_attribute(fights, fighter, 'red_knockdowns', 'blue_knockdowns')
    knockdowns_per_min = remove_nans_at_start_of_carrer(knockdowns_per_min).expanding(2).mean()
    knockdowns_per_min = knockdowns_per_min.to_list()
    
    knockdowns_per_minute = knockdowns_per_minute + knockdowns_per_min

    # Power
    power_rating = list_fighters_attribute(fights, fighter, 'red_power', 'blue_power')
    power_rating = remove_nans_at_start_of_carrer(power_rating).expanding(2).mean()
    power_rating = power_rating.to_list()
    
    power = power + power_rating
    
    # striking Accuracy
    strike_accuracy = list_fighters_attribute(fights, fighter, 'red_striking_accuracy', 'blue_striking_accuracy')
    strike_accuracy = remove_nans_at_start_of_carrer(strike_accuracy).expanding(2).mean()
    strike_accuracy = strike_accuracy.to_list()
    
    striking_accuarcy = striking_accuarcy + strike_accuracy

    # Strikes absorbed per minute
    strike_absorbed = list_fighters_attribute(fights, fighter, 'red_strikes_absorbed_per_minute', 'blue_strikes_absorbed_per_minute')
    strike_absorbed = remove_nans_at_start_of_carrer(strike_absorbed).expanding(2).mean()
    strike_absorbed = strike_absorbed.to_list()
    
    strikes_absorbed_per_min = strikes_absorbed_per_min + strike_absorbed

    # Striking Defense
    striking_def = list_fighters_attribute(fights, fighter, 'red_striking_defense', 'blue_striking_defense')
    striking_def = remove_nans_at_start_of_carrer(striking_def).expanding(2).mean()
    striking_def = striking_def.to_list()
    
    striking_defense = striking_defense + striking_def

    # Submission Attempts per 15mins
    sub_attempts = list_fighters_attribute(fights, fighter, 'red_avg_submissions', 'blue_avg_submissions')
    sub_attempts = remove_nans_at_start_of_carrer(sub_attempts).expanding(2).mean()
    sub_attempts = sub_attempts.to_list()
    
    submission_attempts = submission_attempts + sub_attempts
    
    # Average Number of Takedowns per Minute
    avg_takedowns = list_fighters_attribute(fights, fighter, 'red_avg_takedowns', 'blue_avg_takedowns')
    avg_takedowns = remove_nans_at_start_of_carrer(avg_takedowns).expanding(2).mean()
    avg_takedowns = avg_takedowns.to_list()
    
    average_num_takedowns = average_num_takedowns + avg_takedowns

  return(pd.DataFrame({'Index':rank_indexs,'Fighters':fighters,
                       'Fight_Number':fight_number, 'WinLossRatio':winLossValues,
                       'RingRust':ringRust,'AVG_fight_time':average_fight_time,
                       'Winning_Streak':winning_streak, 'Losing Streak': losing_streak,
                       'Takedown_Defense': takedown_defense, 'Takedown Accuracy':takedown_accuracy,
                       'Strikes_Per_Minute':strikes_per_minute, 'Striking Accuracy':striking_accuarcy,
                       'Strikes_Absorbed_per_Minute':strikes_absorbed_per_min,
                       'Striking Defense':striking_defense, 'Submission Attempts':submission_attempts,
                       'Average_Num_Takedowns':average_num_takedowns, 'knockdows_per_minute':knockdowns_per_minute,
                       'Power_Rating':power}))
  
new_features = feature_engineering_fighter_level_loop(FightsOddsElos)
add_variable_to_split = check_if_each_row_is_either_red_or_blue(new_features, FightsOddsElos)

red = add_variable_to_split[add_variable_to_split.Blue_or_Red == 'Red'].rename(columns = {'Fight_Number':'R_Fight_Number',
                                                         'WinLossRatio':'R_WinLossRatio','RingRust':'R_RingRust',
                                                         'AVG_fight_time':'R_AVG_fight_time','Winning_Streak':'R_Winning_Streak',
                                                          'Losing Streak':'R_Losing_Streak',
                                                          'Takedown_Defense':'R_Takedown_Defense',
                                                          'Takedown Accuracy':'R_Takedown Accuracy',
                                                          'Strikes_Per_Minute':'R_Strikes_Per_Minute',
                                                          'Striking Accuracy':'R_Striking Accuracy',
                                                          'Strikes_Absorbed_per_Minute':'R_Strikes_Absorbed_per_Minute',
                                                          'Striking Defense':'R_Striking Defense',
                                                          'Submission Attempts':'R_Submission Attempts',
                                                          'Average_Num_Takedowns':'R_Average_Num_Takedowns',
                                                          'knockdows_per_minute':'R_knockdows_per_minute',
                                                          'Power_Rating':'R_Power_Rating'}).set_index('Index')
red.drop(['Blue_or_Red','Fighters'], inplace=True,axis=1)

blue = add_variable_to_split[add_variable_to_split.Blue_or_Red == 'Blue'].rename(columns = {'Fight_Number':'B_Fight_Number',
                                                         'WinLossRatio':'B_WinLossRatio','RingRust':'B_RingRust',
                                                         'AVG_fight_time':'B_AVG_fight_time','Winning_Streak':'B_Winning_Streak',
                                                          'Losing Streak':'B_Losing_Streak',
                                                          'Takedown_Defense':'B_Takedown_Defense',
                                                          'Takedown Accuracy':'B_Takedown Accuracy',
                                                          'Strikes_Per_Minute':'B_Strikes_Per_Minute',
                                                          'Striking Accuracy':'B_Striking Accuracy',
                                                          'Strikes_Absorbed_per_Minute':'B_Strikes_Absorbed_per_Minute',
                                                          'Striking Defense':'B_Striking Defense',
                                                          'Submission Attempts':'B_Submission Attempts',
                                                          'Average_Num_Takedowns':'B_Average_Num_Takedowns',
                                                          'knockdows_per_minute':'B_knockdows_per_minute',
                                                          'Power_Rating':'B_Power_Rating'}).set_index('Index')

blue.drop(['Blue_or_Red','Fighters'], inplace=True,axis=1)

Elos_and_features = FightsOddsElos.join(red)
Elos_and_features = Elos_and_features.join(blue)

In [0]:
def extract_stats(row, fighter, red_column, blue_column):
  
  if row.R_fighter == fighter:
    return(row[red_column], 1)

  elif row.B_fighter == fighter:
    return(row[blue_column], 0)


def list_fighters_attribute(df, fighter, red_column, blue_column):
  values = df.apply(lambda x : extract_stats(x, fighter, red_column, blue_column), axis = 1)
  return pd.DataFrame(values.tolist(), columns=['Fighter_Value','Red_or_Blue'], index=values.index)


def replace_shifted_values_back(subset, original_df, fighter, red_column, blue_column):

  values = list_fighters_attribute(subset, fighter, red_column, blue_column)

  # shift values but keep red or blue constant
  values.Fighter_Value = values.Fighter_Value.shift()
  for index, row in values.iterrows():

    if row['Red_or_Blue'] == 1:
      original_df.loc[index, red_column] = row['Fighter_Value']

    else:
      original_df.loc[index, blue_column] = row['Fighter_Value']
    
  return original_df

def Shift_all_features(df):

  Red_fighters  = list(df['R_fighter'].values)
  Blue_fighters = list(df['B_fighter'].values)
  All_fighters  = Red_fighters + Blue_fighters

  unique_fighters = list(set(All_fighters))

  cols_to_shift_red = ['red_fighters_elo','R_WinLossRatio',
                 'R_Winning_Streak','R_Losing_Streak','R_AVG_fight_time',
                 'R_total_title_bouts','R_Takedown_Defense','R_Takedown Accuracy',
                 'R_Strikes_Per_Minute','R_Striking Accuracy','R_Strikes_Absorbed_per_Minute',
                 'R_Striking Defense','R_knockdows_per_minute','R_Submission Attempts',
                 'R_Average_Num_Takedowns','R_Power_Rating','red_skill','wrestling_red_skill',
                 'striking_red_skill', 'g_and_p_red_skill','jiujitsu_red_skill',
                 'grappling_red_skill']
                 
                 
  cols_to_shift_blue  = ['blue_fighters_elo', 'B_WinLossRatio',
                 'B_Winning_Streak','B_Losing_Streak','B_AVG_fight_time',
                 'B_total_title_bouts','B_Takedown_Defense','B_Takedown Accuracy',
                 'B_Strikes_Per_Minute','B_Striking Accuracy','B_Strikes_Absorbed_per_Minute',
                 'B_Striking Defense','B_knockdows_per_minute','B_Submission Attempts',
                 'B_Average_Num_Takedowns','B_Power_Rating','blue_skill','wrestling_blue_skill',
                 'striking_blue_skill', 'g_and_p_blue_skill','jiujitsu_blue_skill',
                 'grappling_blue_skill']

  df = df.sort_values(by='date')

  for fighter in tqdm(unique_fighters):

      fights = df[(df.R_fighter == fighter) | (df.B_fighter == fighter)]

      for red, blue in zip(cols_to_shift_red, cols_to_shift_blue):
        replace_shifted_values_back(subset=fights, 
                                       original_df=df, 
                                       fighter=fighter, 
                                       red_column=red,
                                       blue_column=blue)

  return df

In [0]:
shifted = Shift_all_features(Elos_and_features)
shifted[(shifted.R_fighter == 'Jon Jones') | (shifted.B_fighter == 'Jon Jones')]

,R_fighter,B_fighter,date,Winner,title_bout,weight_class,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_BODY_att,B_avg_BODY_landed,B_avg_CLINCH_att,B_avg_CLINCH_landed,B_avg_DISTANCE_att,B_avg_DISTANCE_landed,B_avg_GROUND_att,B_avg_GROUND_landed,B_avg_HEAD_att,B_avg_HEAD_landed,B_avg_KD,B_avg_LEG_att,B_avg_LEG_landed,B_avg_PASS,B_avg_REV,B_avg_SIG_STR_att,B_avg_SIG_STR_landed,B_avg_SIG_STR_pct,B_avg_SUB_ATT,B_avg_TD_att,B_avg_TD_landed,B_avg_TD_pct,B_avg_TOTAL_STR_att,B_avg_TOTAL_STR_landed,B_longest_win_streak,B_losses,B_avg_opp_BODY_att,B_avg_opp_BODY_landed,B_avg_opp_CLINCH_att,B_avg_opp_CLINCH_landed,B_avg_opp_DISTANCE_att,B_avg_opp_DISTANCE_landed,B_avg_opp_GROUND_att,B_avg_opp_GROUND_landed,B_avg_opp_HEAD_att,B_avg_opp_HEAD_landed,B_avg_opp_KD,B_avg_opp_LEG_att,B_avg_opp_LEG_landed,B_avg_opp_PASS,B_avg_opp_REV,B_avg_opp_SIG_STR_att,B_avg_opp_SIG_STR_landed,B_avg_opp_SIG_STR_pct,B_avg_opp_SUB_ATT,B_avg_opp_TD_att,B_avg_opp_TD_landed,B_avg_opp_TD_pct,B_avg_opp_TOTAL_STR_att,B_avg_opp_TOTAL_STR_landed,B_total_rounds_fought,B_total_time_fought(seconds),B_total_title_bouts,B_win_by_Decision_Majority,B_win_by_Decision_Split,B_win_by_Decision_Unanimous,B_win_by_KO/TKO,B_win_by_Submission,B_win_by_TKO_Doctor_Stoppage,B_wins,B_Stance,B_Height_cms,B_Reach_cms,B_Weight_lbs,R_current_lose_streak,R_current_win_streak,R_draw,R_avg_BODY_att,R_avg_BODY_landed,R_avg_CLINCH_att,R_avg_CLINCH_landed,R_avg_DISTANCE_att,R_avg_DISTANCE_landed,R_avg_GROUND_att,R_avg_GROUND_landed,R_avg_HEAD_att,R_avg_HEAD_landed,R_avg_KD,R_avg_LEG_att,R_avg_LEG_landed,R_avg_PASS,R_avg_REV,R_avg_SIG_STR_att,R_avg_SIG_STR_landed,R_avg_SIG_STR_pct,R_avg_SUB_ATT,R_avg_TD_att,R_avg_TD_landed,R_avg_TD_pct,R_avg_TOTAL_STR_att,R_avg_TOTAL_STR_landed,...,R_GROUND,B_CLINCH,R_CLINCH,B_DISTANCE,R_DISTANCE,B_LEG,R_LEG,B_BODY,R_BODY,B_HEAD,R_HEAD,B_REV,R_REV,B_PASS,R_PASS,B_SUB_ATT,R_SUB_ATT,B_TD_pct,R_TD_pct,B_TD,R_TD,B_TOTAL_STR.,R_TOTAL_STR.,B_SIG_STR_pct,R_SIG_STR_pct,B_SIG_STR.,R_SIG_STR.,B_KD,R_KD,red_strikes_per_minute,blue_strikes_per_minute,red_striking_accuracy,blue_striking_accuracy,red_avg_takedowns,blue_avg_takedowns,red_td_accuracy,blue_td_accuracy,red_td_defense,blue_td_defense,red_strikes_absorbed_per_minute,blue_strikes_absorbed_per_minute,red_striking_defense,blue_striking_defense,red_avg_submissions,blue_avg_submissions,red_knockdowns,blue_knockdowns,red_power,blue_power,red_fighters_elo,blue_fighters_elo,Who_Won_at_Wrestling,Who_Won_at_Striking,Who_Won_at_Ground&Pound,Who_Won_at_JiuJitsu,Who_Won_at_Grappling,red_skill,blue_skill,wrestling_red_skill,wrestling_blue_skill,striking_red_skill,striking_blue_skill,g_and_p_red_skill,g_and_p_blue_skill,jiujitsu_red_skill,jiujitsu_blue_skill,grappling_red_skill,grappling_blue_skill,R_Fight_Number,R_WinLossRatio,R_RingRust,R_AVG_fight_time,R_Winning_Streak,R_Losing_Streak,R_Takedown_Defense,R_Takedown Accuracy,R_Strikes_Per_Minute,R_Striking Accuracy,R_Strikes_Absorbed_per_Minute,R_Striking Defense,R_Submission Attempts,R_Average_Num_Takedowns,R_knockdows_per_minute,R_Power_Rating,B_Fight_Number,B_WinLossRatio,B_RingRust,B_AVG_fight_time,B_Winning_Streak,B_Losing_Streak,B_Takedown_Defense,B_Takedown Accuracy,B_Strikes_Per_Minute,B_Striking Accuracy,B_Strikes_Absorbed_per_Minute,B_Striking Defense,B_Submission Attempts,B_Average_Num_Takedowns,B_knockdows_per_minute,B_Power_Rating
4271,Jon Jones,Andre Gusmao,2008-08-09,Red,False,Light Heavyweight,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Orthodox,182.88,NaN,205.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3 of 5,15 of 41,26 of 51,19 of 43,13 of 46,12 of 15,1 of 5,11 of 20,16 of 24,11 of 49,25 of 73,0,0,0,2,0,0,0%,40%,0 of 1,2 of 5,46 of 98,68 of 129,40%,41%,34 of 84,42 of 102,0,0,6.800

In [0]:
sub = shifted[['date','R_fighter','B_fighter','R_Fight_Number',
               'B_Fight_Number', 'red_fighters_elo', 'blue_fighters_elo',
               'red_skill','blue_skill', 'R_win_by_Decision_Majority',
               'R_win_by_Decision_Split','R_win_by_Decision_Unanimous',
               'R_win_by_KO/TKO', 'R_win_by_Submission', 'R_win_by_TKO_Doctor_Stoppage']]
sub[(sub.R_fighter == 'Jon Jones') | (sub.B_fighter == 'Jon Jones')]

,date,R_fighter,B_fighter,R_Fight_Number,B_Fight_Number,red_fighters_elo,blue_fighters_elo,red_skill,blue_skill,R_win_by_Decision_Majority,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage
4271,2008-08-09,Jon Jones,Andre Gusmao,1.0,1.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
4180,2009-01-31,Jon Jones,Stephan Bonnar,2.0,3.0,1236.250000,1268.731489,29.395832,31.957181,0.0,0.0,1.0,0.0,0.0,0.0
4008,2009-12-05,Matt Hamill,Jon Jones,6.0,3.0,1259.088427,1275.879146,27.007014,33.909214,0.0,0.0,1.0,5.0,0.0,0.0
3956,2010-03-21,Brandon Vera,Jon Jones,8.0,4.0,1205.272926,1262.775521,24.231346,28.697785,0.0,0.0,2.0,4.0,1.0,0.0
3868,2010-08-01,Jon Jones,Vladimir Matyushenko,5.0,3.0,1293.080124,1250.232902,30.311675,34.979661,0.0,0.0,2.0,1.0,1.0,0.0
3737,2011-02-05,Jon Jones,Ryan Bader,6.0,4.0,1324.882147,1301.605293,32.945964,34.436510,0.0,0.0,2.0,2.0,1.0,0.0
3703,2011-03-19,Mauricio Rua,Jon Jones,6.0,7.0,1266.644708,1358.707163,32.693252,34.761194,0.0,0.0,0.0,3.0,0.0,0.0
3493,2011-12-10,Jon Jones,Lyoto Machida,8.0,8.0,1385.570414,1351.724920,36.019568,35.878484,0.0,0.0,2.0,3.0,3.0,0.0
3389,2012-04-21,Jon Jones,Rashad Evans,9.0,9.0,1418.300242,1356.406477,37.419576,35.832435,0.0,0.0,2.0,3.0,4.0,0.0
3252,2012-09-22,Jon Jones,Vitor Belfort,10.0,5.0,1448.159958,1300.447208,38.493637,35.455897,0.0,0.0,3.0,3.0,4.0,0.0


In [0]:
def Normalize_different_wins(df):

  my_copy = df.copy()
  red_columns = ['R_win_by_Decision_Majority','R_win_by_Decision_Split','R_win_by_Decision_Unanimous',
   'R_win_by_KO/TKO', 'R_win_by_Submission', 'R_win_by_TKO_Doctor_Stoppage']

  blue_columns = ['B_win_by_Decision_Majority','B_win_by_Decision_Split','B_win_by_Decision_Unanimous',
   'B_win_by_KO/TKO', 'B_win_by_Submission', 'B_win_by_TKO_Doctor_Stoppage']

  for red, blue in zip(red_columns, blue_columns):

    my_copy[red]  = my_copy[red]/my_copy['R_Fight_Number']
    my_copy[blue] = my_copy[blue]/my_copy['B_Fight_Number']

  return(my_copy)

column_subset = shifted[['R_fighter','B_fighter','Average_Odds_f1', 'Average_Odds_f2','date','title_bout','win_by','weight_class',
                            'red_fighters_elo','blue_fighters_elo','red_Fighter_Odds','blue_Fighter_Odds','Winner',
                            'R_Fight_Number','R_Stance', 'R_Height_cms', 'R_Reach_cms', 'R_age', 'R_WinLossRatio',
                            'R_RingRust','R_Winning_Streak','R_Losing_Streak','R_AVG_fight_time','R_total_title_bouts',
                            'R_Takedown_Defense', 'R_Takedown Accuracy','R_Strikes_Per_Minute','R_Striking Accuracy',
                            'R_Strikes_Absorbed_per_Minute','R_Striking Defense','R_knockdows_per_minute','R_Submission Attempts',
                            'R_Average_Num_Takedowns','R_win_by_Decision_Majority','R_win_by_Decision_Split','R_win_by_Decision_Unanimous',
                            'R_win_by_KO/TKO', 'R_win_by_Submission', 'R_win_by_TKO_Doctor_Stoppage','R_Power_Rating','red_skill',
                            'wrestling_red_skill','striking_red_skill','g_and_p_red_skill', 'jiujitsu_red_skill', 'grappling_red_skill',
                            'B_Fight_Number',
                            'B_Stance','B_Height_cms','B_Reach_cms', 'B_age','B_WinLossRatio','B_RingRust','B_Winning_Streak',
                            'B_Losing_Streak','B_AVG_fight_time', 'B_total_title_bouts','B_Takedown_Defense', 'B_Takedown Accuracy',
                            'B_Strikes_Per_Minute','B_Striking Accuracy','B_Strikes_Absorbed_per_Minute','B_Striking Defense',
                            'B_knockdows_per_minute','B_Submission Attempts','B_Average_Num_Takedowns','B_win_by_Decision_Majority',
                            'B_win_by_Decision_Split','B_win_by_Decision_Unanimous','B_win_by_KO/TKO','B_win_by_Submission',
                            'B_win_by_TKO_Doctor_Stoppage','B_Power_Rating','blue_skill', 'wrestling_blue_skill', 'striking_blue_skill',
                            'g_and_p_blue_skill', 'jiujitsu_blue_skill', 'grappling_blue_skill']]

new_column_subset = Normalize_different_wins(column_subset)

In [0]:
column_subset

,R_fighter,B_fighter,Average_Odds_f1,Average_Odds_f2,date,title_bout,win_by,weight_class,red_fighters_elo,blue_fighters_elo,red_Fighter_Odds,blue_Fighter_Odds,Winner,R_Fight_Number,R_Stance,R_Height_cms,R_Reach_cms,R_age,R_WinLossRatio,R_RingRust,R_Winning_Streak,R_Losing_Streak,R_AVG_fight_time,R_total_title_bouts,R_Takedown_Defense,R_Takedown Accuracy,R_Strikes_Per_Minute,R_Striking Accuracy,R_Strikes_Absorbed_per_Minute,R_Striking Defense,R_knockdows_per_minute,R_Submission Attempts,R_Average_Num_Takedowns,R_win_by_Decision_Majority,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Power_Rating,red_skill,wrestling_red_skill,striking_red_skill,g_and_p_red_skill,jiujitsu_red_skill,grappling_red_skill,B_Fight_Number,B_Stance,B_Height_cms,B_Reach_cms,B_age,B_WinLossRatio,B_RingRust,B_Winning_Streak,B_Losing_Streak,B_AVG_fight_time,B_total_title_bouts,B_Takedown_Defense,B_Takedown Accuracy,B_Strikes_Per_Minute,B_Striking Accuracy,B_Strikes_Absorbed_per_Minute,B_Striking Defense,B_knockdows_per_minute,B_Submission Attempts,B_Average_Num_Takedowns,B_win_by_Decision_Majority,B_win_by_Decision_Split,B_win_by_Decision_Unanimous,B_win_by_KO/TKO,B_win_by_Submission,B_win_by_TKO_Doctor_Stoppage,B_Power_Rating,blue_skill,wrestling_blue_skill,striking_blue_skill,g_and_p_blue_skill,jiujitsu_blue_skill,grappling_blue_skill
4461,Dustin Hazelett,Stevie Lynch,-250.0,225.0,2007-06-16,False,Submission,Welterweight,NaN,NaN,0.714286,0.307692,Red,1.0,Orthodox,185.42,193.04,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Orthodox,177.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4455,Rich Franklin,Yushin Okami,-210.0,180.0,2007-06-16,False,Decision - Unanimous,Middleweight,NaN,NaN,0.677419,0.357143,Red,1.0,Southpaw,185.42,190.50,32.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,5.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Southpaw,187.96,190.50,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,2.0,2.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4456,Forrest Griffin,Hector Ramirez,-200.0,200.0,2007-06-16,False,Decision - Unanimous,Light Heavyweight,NaN,NaN,0.666667,0.333333,Red,1.0,Orthodox,190.50,195.58,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,2.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Orthodox,182.88,NaN,31.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4460,Eddie Sanchez,Colin Robinson,-200.0,190.0,2007-06-16,False,KO/TKO,Heavyweight,NaN,NaN,0.666667,0.344828,Red,1.0,Orthodox,187.96,187.96,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Orthodox,193.04,NaN,38.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4458,Ed Herman,Scott Smith,-160.0,135.0,2007-06-16,False,Submission,Middleweight,NaN,NaN,0.615385,0.425532,Red,1.0,Orthodox,185.42,195.58,26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Orthodox,182.88,177.80,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,Mayra Bueno Silva,Maryna Moroz,175.0,150.0,2020-03-14,False,Decision - Unanimous,Women's Flyweight,NaN,1204.062645,0.363636,0.400000,Blue,1.0,Orthodox,167.64,167.64,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0

In [0]:
column_subset.to_csv('drive/My Drive/UFC/ufc_processed_data_v2.csv',index=False)